### Expected Errors Testing

#### Usman Asad, Faisal Jaffer, Aleem Haq. Modified in April 2021

This notebook contains a series of procedures to test the expected errors associated with the map.

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab
import time

In [ ]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    def time_(s): 
        print(time.time() * 1000)
        return time.time() * 1000
    def key_err(s):
        raise Exception("Key does not exist")
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'time': time_,'err': key_err}})
    
def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))
    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [ ]:
# Testing to see that an error is thrown when insertion is attempted but the map is not yet initialized

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.insert(2,4)
    write(e.get(2))
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
# Testing to see if error is thrown when inserting wrong type into the map

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.init
    write(e.len())
    e.insert(1, true)
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
# Getting a value that does not exist

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.init
    a ← time()
    write(e.len())
    write(e.get(13233))
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
# Testing to see wrong key type (Key expected to be Int, Bool, Set)

In [ ]:
compileString("""
program bitsets
  var e: map <[0..5] → integer, integer>
  var a, b: integer
    e.init
    a ← time()
    write(e.len())
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
# Testing to see value accepts all types

In [ ]:
compileString("""
program bitsets
  var e: map <integer, [0..5] → integer>
  var a, b: integer
    e.init
    a ← time()
    write(e.len())
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
## Testing if bad type error is thrown for type mismatch for key in map

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.init
    if true in e then write(1)
    else write(0)
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
## Testing basic syntax  

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.init
    write(e.len()
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.init()
    write(e.len())
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
compileString("""
program bitsets
  var e: map <integer, integer>
  var a, b: integer
    e.init()
    write(e.len)
""", 'test.wat', target = 'wat')
!wat2wasm test.wat
runpywasm("test.wasm")

In [ ]:
# Clean up
!rm test*